<a href="https://colab.research.google.com/github/matheusprandini/dnns-game-events/blob/feat%2Fadding-docs/src/I3D_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
import glob
import math
import os
import numpy as np
import random
import shutil
import zipfile

from google.colab import drive
from tensorflow.keras.applications import (
    MobileNetV2, 
    ResNet50V2, 
    VGG16)
from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint)
from tensorflow.keras.layers import (
    Activation,
    BatchNormalization, 
    Conv2D, 
    Conv3D, 
    Dense,
    Dropout, 
    Flatten,
    GlobalAveragePooling2D, 
    GRU,  
    Input, 
    LSTM, 
    MaxPooling2D, 
    Reshape)
from tensorflow.keras.models import (
    Model, 
    Sequential,
    load_model)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import (
    Sequence, 
    to_categorical)

## Loading Datasets

In [ ]:
drive.mount('/content/drive',force_remount=True)

### Mario Datasets

#### MobileNetV2 - Extracted Features from Chunks of 6 frames

In [ ]:
driveFilePath = "/content/drive/MyDrive/Datasets/Mario12EventsDataset_Chunk6_135Videos_MobileNetV2_Train.zip"
zip_ref = zipfile.ZipFile(driveFilePath, 'r')
zip_ref.extractall("/content/chunk-dataset/mobilenetv2/")
zip_ref.close()

inputDataset = "/content/chunk-dataset/mobilenetv2/Mario12EventsDataset_Chunk6_135Videos_MobileNetV2_Train/"
trainDataset = "/content/chunk-dataset/mobilenetv2/Mario12Events/train/"

#### ResNet50V2 - Extracted Features from Chunks of 6 frames

In [ ]:
driveFilePath = "/content/drive/MyDrive/Datasets/Mario12EventsDataset_Chunk6_135Videos_ResNet50V2_Train.zip"
zip_ref = zipfile.ZipFile(driveFilePath, 'r')
zip_ref.extractall("/content/chunk-dataset/resnet50v2/")
zip_ref.close()

inputDataset = "/content/chunk-dataset/resnet50v2/Mario12EventsDataset_Chunk6_135Videos_ResNet50V2_Train/"
trainDataset = "/content/chunk-dataset/resnet50v2/Mario12Events/train/"

#### VGG16 - Extracted Features from Chunks of 6 frames

In [ ]:
driveFilePath = "/content/drive/MyDrive/Datasets/Mario12EventsDataset_Chunk6_135Videos_VGG16_Train.zip"
zip_ref = zipfile.ZipFile(driveFilePath, 'r')
zip_ref.extractall("/content/chunk-dataset/vgg16/")
zip_ref.close()

inputDataset = "/content/chunk-dataset/vgg16/Mario12EventsDataset_Chunk6_135Videos_VGG16_Train/"
trainDataset = "/content/chunk-dataset/vgg16/Mario12Events/train/"

#### AlexNet - Extracted Features from Chunks of 6 frames

In [ ]:
driveFilePath = "/content/drive/MyDrive/Datasets/Mario12EventsDataset_Chunk6_135Videos_AlexNet_Train.zip"
zip_ref = zipfile.ZipFile(driveFilePath, 'r')
zip_ref.extractall("/content/chunk-dataset/alexnet/")
zip_ref.close()

inputDataset = "/content/chunk-dataset/alexnet/Mario12EventsDataset_Chunk6_135Videos_AlexNet_Train/"
trainDataset = "/content/chunk-dataset/alexnet/Mario12Events/train/"

#### MobileNetV2 - Extracted Features from Individual frames

In [ ]:
driveFilePath = "/content/drive/MyDrive/Datasets/Mario12EventsDataset_IndividualFrames_135Videos_MobileNetV2_Train.zip"
zip_ref = zipfile.ZipFile(driveFilePath, 'r')
zip_ref.extractall("/content/individual-frames/mobilenetv2/")
zip_ref.close()

inputDataset = "/content/individual-frames/mobilenetv2/Mario12EventsDataset_IndividualFrames_135Videos_MobileNetV2_Train/"
trainDataset = "/content/individual-frames/mobilenetv2/Mario12Events/train/"

#### ResNet50V2 - Extracted Features from Individual frames

In [ ]:
driveFilePath = "/content/drive/MyDrive/Datasets/Mario12EventsDataset_IndividualFrames_135Videos_ResNet50V2_Train.zip"
zip_ref = zipfile.ZipFile(driveFilePath, 'r')
zip_ref.extractall("/content/individual-frames/resnet50v2/")
zip_ref.close()

inputDataset = "/content/individual-frames/resnet50v2/Mario12EventsDataset_IndividualFrames_135Videos_ResNet50V2_Train/"
trainDataset = "/content/individual-frames/resnet50v2/Mario12Events/train/"

#### VGG16 - Extracted Features from Individual frames

In [ ]:
driveFilePath = "/content/drive/MyDrive/Datasets/Mario12EventsDataset_IndividualFrames_135Videos_VGG16_Train.zip"
zip_ref = zipfile.ZipFile(driveFilePath, 'r')
zip_ref.extractall("/content/individual-frames/vgg16/")
zip_ref.close()

inputDataset = "/content/individual-frames/vgg16/Mario12EventsDataset_IndividualFrames_135Videos_VGG16_Train/"
trainDataset = "/content/individual-frames/vgg16/Mario12Events/train/"

#### AlexNet - Extracted Features from Individual frames

In [ ]:
driveFilePath = "/content/drive/MyDrive/Datasets/Mario12EventsDataset_IndividualFrames_135Videos_AlexNet_Train.zip"
zip_ref = zipfile.ZipFile(driveFilePath, 'r')
zip_ref.extractall("/content/individual-frames/alexnet/")
zip_ref.close()

inputDataset = "/content/individual-frames/alexnet/Mario12EventsDataset_IndividualFrames_135Videos_AlexNet_Train/"
trainDataset = "/content/individual-frames/alexnet/Mario12Events/train/"

#### Raw Individual Frames - End to End AlexNet and MobileNetV2

In [ ]:
driveFilePath = "/content/drive/MyDrive/Datasets/Mario12EventsDataset_IndividualFrames_135Videos_RawImages_Train.zip"
zip_ref = zipfile.ZipFile(driveFilePath, 'r')
zip_ref.extractall("/content/individual-frames/rawimages/")
zip_ref.close()

inputDataset = "/content/individual-frames/rawimages/Mario12EventsDataset_IndividualFrames_135Videos_RawImages_Train/"
trainDataset = "/content/individual-frames/rawimages/Mario12Events/train/"

## Feature Extractors

### MobileNetV2

In [ ]:
class MobileNetV2Model():

    def build_model(self, weights, includeTop, inputShape):
        baseModel = MobileNetV2(weights=weights, include_top=includeTop, input_shape=inputShape)

        model = Model(
            inputs=baseModel.input,
            outputs=baseModel.get_layer('global_average_pooling2d_3').output
        )
        
        return model

### ResNet50V2

In [ ]:
class ResNet50V2Model():

    def build_model(self, weights, includeTop, inputShape):
        baseModel = ResNet50V2(weights=weights, include_top=includeTop, input_shape=inputShape)

        model = Model(
            inputs=baseModel.input,
            outputs=baseModel.get_layer('avg_pool').output
        )
        
        return model

### VGG16

In [ ]:
class VGG16Model():

    def build_model(self, weights, includeTop, inputShape):
        baseModel = VGG16(weights=weights, include_top=includeTop, input_shape=inputShape)

        model = Model(
            inputs=baseModel.input,
            outputs=baseModel.get_layer('fc2').output
        )
        
        return model

### AlexNet

In [ ]:
class AlexNetModel():

    def build_model(self, weights, includeTop, inputShape):
        model = Sequential()
        # 1nd Convolutional Layer
        model.add(Conv2D(filters=96, input_shape=inputShape, kernel_size=(11,11), strides=(4,4), padding='valid'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
        model.add(BatchNormalization())
        # 2nd Convolutional Layer
        model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
        model.add(BatchNormalization())
        # 3rd Convolutional Layer
        model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        # 4th Convolutional Layer
        model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        # 5th Convolutional Layer
        model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
        model.add(Activation('relu'))
        model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
        model.add(BatchNormalization())
        print(model.summary())
        return model

### Factory

In [ ]:
class FeatureExtractorFactory():
    modelNameToFeatureExtractorModel = {
        "MobileNetV2": MobileNetV2Model(),
        "ResNet50V2": ResNet50V2Model(),
        "VGG16": VGG16Model(),
        "AlexNet": AlexNetModel()
    }

    @staticmethod
    def get_model(name, weights="imagenet", includeTop=True, inputShape=(224,224,3)):
        if name not in FeatureExtractorFactory.modelNameToFeatureExtractorModel:
            return None
        return FeatureExtractorFactory.modelNameToFeatureExtractorModel[name].build_model(weights, includeTop, inputShape)

In [ ]:
model = FeatureExtractorFactory.get_model("VGG16")
model.summary()

## Classifiers

In [ ]:
class TRN():

    def __init__(self):
        self.model = None

    def compile_model(self, learningRate):
        self.model.compile(
            loss="categorical_crossentropy",
            optimizer=Adam(learning_rate=learningRate),
            metrics=["accuracy"])

    def train_model(self, trainGenerator, validationGenerator, numEpochs=100, modelName="test"):
        checkpointer = ModelCheckpoint(
            filepath=modelName + ".h5",
            verbose=1,
            save_best_only=True)
        
        earlyStopper = EarlyStopping(patience=20)

        self.model.fit(
            x=trainGenerator, 
            validation_data=validationGenerator, 
            epochs=numEpochs,
            callbacks=[checkpointer, earlyStopper])

### GRU

In [ ]:
class TRNGRU(TRN):

    def build_model(self, numTimesteps, numFeatures, numHiddenNodes=1280, numClasses=12):
        self.model = Sequential()
        self.model.add(GRU(numHiddenNodes, input_shape=(numTimesteps, numFeatures), dropout=0.2))
        self.model.add(Dense(numClasses, activation="softmax"))
        print(self.model.summary())

### LSTM

In [ ]:
class TRNLSTM(TRN):

    def build_model(self, numTimesteps, numFeatures, numHiddenNodes=1280, numClasses=12):
        self.model = Sequential()
        self.model.add(LSTM(numHiddenNodes, input_shape=(numTimesteps, numFeatures), dropout=0.2))
        self.model.add(Dense(numClasses, activation="softmax"))
        print(self.model.summary())

### Dense - Full Connected Layers (FCL) 

Neural network based on the classifier layers of AlexNet.

In [ ]:
class TRNDense(TRN):

    def build_model(self, numTimesteps, numFeatures, numHiddenNodes=1280, numClasses=12):
        self.model = Sequential()
        self.model.add(Dense(4096, input_shape=(numTimesteps*numFeatures,)))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.4))
        self.model.add(BatchNormalization())
        # 2nd Dense Layer
        self.model.add(Dense(4096))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.4))
        self.model.add(BatchNormalization())
        # 3rd Dense Layer
        self.model.add(Dense(1000))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.4))
        self.model.add(BatchNormalization())
        # Output Layer
        self.model.add(Dense(numClasses))
        self.model.add(Activation('softmax'))
        print(self.model.summary())

### End to End - AlexNet

In [ ]:
class TRNAlexNet(TRN):

    def build_model(self, numTimesteps=None, numFeatures=None, numHiddenNodes=None, numClasses=12):
        self.model = Sequential()
        # 1nd Convolutional Layer
        self.model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11), strides=(4,4), padding='valid'))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        #self.model.add(BatchNormalization())
        # 2nd Convolutional Layer
        self.model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='same'))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        #self.model.add(BatchNormalization())
        # 3rd Convolutional Layer
        self.model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
        self.model.add(Activation('relu'))
        #self.model.add(BatchNormalization())
        # 4th Convolutional Layer
        self.model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='same'))
        self.model.add(Activation('relu'))
        #self.model.add(BatchNormalization())
        # 5th Convolutional Layer
        self.model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='same'))
        self.model.add(Activation('relu'))
        self.model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
        #self.model.add(BatchNormalization())
        # Passing it to a dense layer
        self.model.add(Flatten())
        # 1st Dense Layer
        self.model.add(Dense(4096, input_shape=(224*224*3,)))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.4))
        #self.model.add(BatchNormalization())
        # 2nd Dense Layer
        self.model.add(Dense(4096))
        self.model.add(Activation('relu'))
        self.model.add(Dropout(0.4))
        #self.model.add(BatchNormalization())
        # 3rd Dense Layer
        # self.model.add(Dense(1000))
        # self.model.add(Activation('relu'))
        # self.model.add(Dropout(0.4))
        #self.model.add(BatchNormalization())
        # Output Layer
        self.model.add(Dense(numClasses))
        self.model.add(Activation('softmax'))
        print(self.model.summary())

### End to end - MobileNet

In [ ]:
class TRNMobileNet(TRN):

    def build_model(self, numTimesteps=None, numFeatures=None, numHiddenNodes=None, numClasses=12):
        base_model = MobileNetV2(
            weights=None,
            include_top=False,
            input_shape=(224, 224, 3)
        )

        inputs = Input(shape=(224, 224, 3))
        x = base_model(inputs)
        x = GlobalAveragePooling2D()(x)
        logits = Dense(numClasses)(x)
        outputs = Activation("softmax")(logits)

        self.model = Model(inputs, outputs)
        print(self.model.summary())

### Factory

In [ ]:
class TRNFactory():
    typeToModel = {
        "lstm": TRNLSTM,
        "gru": TRNGRU,
        "alexnet": TRNAlexNet,
        "dense": TRNDense,
        'mobilenet': TRNMobileNet
    }

    @staticmethod
    def get_model(typeName):
        if typeName not in TRNFactory.typeToModel:
            return None
        return TRNFactory.typeToModel[typeName]()

## Generating Training/Testing Data

### Data Generator

In [ ]:
class DataGenerator(Sequence):

    def __init__(self, listIDs, labels, dim=(1,6,1280,1), batchSize=32, numClasses=13, shuffle=True):
        self.listIDs = listIDs
        self.labels = labels
        self.batchSize = batchSize
        self.dim = dim
        self.numClasses = numClasses
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.listIDs) / self.batchSize))

    def __getitem__(self, index):
        indexes = self.indexes[index*self.batchSize:(index+1)*self.batchSize]
        listIDsBatch = [self.listIDs[k] for k in indexes]
        X, y = self.__data_generation(listIDsBatch)

        return X, y

    def on_epoch_end(self):
        self.indexes = np.arange(len(self.listIDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, listIDsBatch):
        X = np.empty((self.batchSize, *self.dim))
        y = np.empty((self.batchSize), dtype=int)

        for i, ID in enumerate(listIDsBatch):
            X[i,] = np.load(ID).reshape(self.dim)
            y[i] = self.labels[ID]
          
        return X, to_categorical(y, num_classes=self.numClasses)

### Data Partitions

In [ ]:
class DataPartitions():
    
    classLabels = {'EventBump': 0, 'EventCollect': 1, 'EventFallKill': 2, 'EventFireKill': 3,
                   'EventHurt': 4, 'EventJump': 5, 'EventKick': 6, 'EventLand': 7,
                   'EventLose': 8, 'EventShellKill': 9, 'EventStompKill': 10, 'EventWin': 11}

    @staticmethod
    def create_partitions_and_labels_cv_train(foldsPath, valIndex):
        partition = []
        labels = {}

        for foldIndex in foldsPath.keys():

            if foldIndex == valIndex:
                continue

            dataPath = foldsPath[foldIndex]
            allFiles = os.listdir(dataPath)

            for classFolder in allFiles:
                if classFolder not in DataPartitions.classLabels:
                    continue
                allVideosPath = dataPath + classFolder + "/"
                allVideos = os.listdir(allVideosPath)
                for video in allVideos:
                    chunkPath = allVideosPath + video
                    partition.append(chunkPath)
                    labels[chunkPath] = DataPartitions.classLabels[classFolder]

        return partition, labels

    @staticmethod
    def create_partitions_and_labels_cv_val(foldsPath, valIndex):
        partition = []
        labels = {}

        dataPath = foldsPath[valIndex]
        allFiles = os.listdir(dataPath)

        for classFolder in allFiles:
            if classFolder not in DataPartitions.classLabels:
                continue
            allVideosPath = dataPath + classFolder + "/"
            allVideos = os.listdir(allVideosPath)
            for video in allVideos:
                chunkPath = allVideosPath + video
                partition.append(chunkPath)
                labels[chunkPath] = DataPartitions.classLabels[classFolder]

        return partition, labels

### Splitting Data - 5-Fold Cross Validation

In [ ]:
Kfolds = 5

folds = {}
for k in range(Kfolds):
    folds[k] = trainDataset + "cv" + str(k) + "/"

def create_data():
    subDirs = glob.glob(inputDataset + "*")
    random.shuffle(subDirs)

    for subDir in subDirs:
        eventName = subDir.split(inputDataset)[-1]
        
        allChunks = glob.glob(subDir + "/*")

        for i in range(len(allChunks)):
            chunkPath = allChunks[i]
            chunkName = chunkPath.split('/')[-1]
            
            foldIndex = i % Kfolds
            newDir = folds[foldIndex] + eventName

            destFilepath = newDir + "/" + chunkName
            os.makedirs(newDir, exist_ok=True)
            shutil.move(chunkPath, destFilepath)

create_data()

## Training Models

In [ ]:
def train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim
):
    epochs = 100
    learningRate = 0.0005

    for foldIndex in range(5):

        finalModelFilename = "trn_" + featureExtractorName + "_" + trnType + "_" + str(numTimesteps) + "_" + str(foldIndex)

        partitions = {}
        labels = {}

        partitions["train"], labels["train"] = DataPartitions.create_partitions_and_labels_cv_train(folds, foldIndex)
        partitions["validation"], labels["validation"] = DataPartitions.create_partitions_and_labels_cv_val(folds, foldIndex)

        trainGenerator = DataGenerator(partitions["train"], labels["train"], dim=input_dim, numClasses=numClasses)
        validationGenerator = DataGenerator(partitions["validation"], labels["validation"], dim=input_dim, numClasses=numClasses)

        trn = TRNFactory.get_model(trnType)
        trn.build_model(numTimesteps, numFeatures, numHiddenNodes, numClasses)
        trn.compile_model(learningRate)
        trn.train_model(trainGenerator, validationGenerator, epochs, finalModelFilename)

### Model 1 - MobileNetV2 + LSTM + Chunk

In [ ]:
numTimesteps = 6
numFeatures = 1280
numHiddenNodes = 1280
numClasses = 12
featureExtractorName = "mobilenetv2"
trnType = "lstm"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

In [ ]:
best_model = load_model("/content/trn_mobilenetv2_lstm_6_0.h5")

partitions = {}
labels = {}
best_fold_index = 0

partitions["validation"], labels["validation"] = DataPartitions.create_partitions_and_labels_cv_val(folds, best_fold_index)
validationGenerator = DataGenerator(partitions["validation"], labels["validation"], dim=input_dim, numClasses=numClasses)

best_model.evaluate(validationGenerator)

### Model 2 - MobileNetV2 + LSTM + Frame

In [ ]:
numTimesteps = 1
numFeatures = 1280
numHiddenNodes = 1280
numClasses = 12
featureExtractorName = "mobilenetv2"
trnType = "lstm"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 3 - MobileNetV2 + GRU + Chunk

In [ ]:
numTimesteps = 6
numFeatures = 1280
numHiddenNodes = 1280
numClasses = 12
featureExtractorName = "mobilenetv2"
trnType = "gru"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 4 - MobileNetV2 + GRU + Frame

In [ ]:
numTimesteps = 1
numFeatures = 1280
numHiddenNodes = 1280
numClasses = 12
featureExtractorName = "mobilenetv2"
trnType = "gru"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 5 - MobileNetV2 + Dense + Chunk

In [ ]:
numTimesteps = 6
numFeatures = 1280
numHiddenNodes = 1280
numClasses = 12
featureExtractorName = "mobilenetv2"
trnType = "dense"
input_dim = (numTimesteps*numFeatures,)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 6 - MobileNetV2 + Dense + Frame

In [ ]:
numTimesteps = 1
numFeatures = 1280
numHiddenNodes = 1280
numClasses = 12
featureExtractorName = "mobilenetv2"
trnType = "dense"
input_dim = (numTimesteps*numFeatures,)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 7 - ResNet50V2 + LSTM + Chunk

In [ ]:
numTimesteps = 6
numFeatures = 2048
numHiddenNodes = 2048
numClasses = 12
featureExtractorName = "resnet50v2"
trnType = "lstm"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 8 - ResNet50V2 + LSTM + Frame

In [ ]:
numTimesteps = 1
numFeatures = 2048
numHiddenNodes = 2048
numClasses = 12
featureExtractorName = "resnet50v2"
trnType = "lstm"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 9 - ResNet50V2 + GRU + Chunk

In [ ]:
numTimesteps = 6
numFeatures = 2048
numHiddenNodes = 2048
numClasses = 12
featureExtractorName = "resnet50v2"
trnType = "gru"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 10 - ResNet50V2 + GRU + Frame

In [ ]:
numTimesteps = 1
numFeatures = 2048
numHiddenNodes = 2048
numClasses = 12
featureExtractorName = "resnet50v2"
trnType = "gru"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 11 - ResNet50V2 + Dense + Chunk

In [ ]:
numTimesteps = 6
numFeatures = 2048
numHiddenNodes = 2048
numClasses = 12
featureExtractorName = "resnet50v2"
trnType = "dense"
input_dim = (numTimesteps*numFeatures,)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 12 - ResNet50V2 + Dense + Frame

In [ ]:
numTimesteps = 1
numFeatures = 2048
numHiddenNodes = 2048
numClasses = 12
featureExtractorName = "resnet50v2"
trnType = "dense"
input_dim = (numTimesteps*numFeatures,)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 13 - VGG16 + LSTM + Chunk

In [ ]:
numTimesteps = 6
numFeatures = 4096
numHiddenNodes = 4096
numClasses = 12
featureExtractorName = "vgg16"
trnType = "lstm"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 14 - VGG16 + LSTM + Frame

In [ ]:
numTimesteps = 1
numFeatures = 4096
numHiddenNodes = 4096
numClasses = 12
featureExtractorName = "vgg16"
trnType = "lstm"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 15 - VGG16 + GRU + Chunk

In [ ]:
numTimesteps = 6
numFeatures = 4096
numHiddenNodes = 4096
numClasses = 12
featureExtractorName = "vgg16"
trnType = "gru"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 16 - VGG16 + GRU + Frame

In [ ]:
numTimesteps = 1
numFeatures = 4096
numHiddenNodes = 4096
numClasses = 12
featureExtractorName = "vgg16"
trnType = "gru"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 17 - VGG16 + Dense + Chunk

In [ ]:
numTimesteps = 6
numFeatures = 4096
numHiddenNodes = 4096
numClasses = 12
featureExtractorName = "vgg16"
trnType = "dense"
input_dim = (numTimesteps*numFeatures,)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 18 - VGG16 + Dense + Frame

In [ ]:
numTimesteps = 1
numFeatures = 4096
numHiddenNodes = 4096
numClasses = 12
featureExtractorName = "vgg16"
trnType = "dense"
input_dim = (numTimesteps*numFeatures,)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 19 - AlexNet + LSTM + Chunk

In [ ]:
numTimesteps = 6
numFeatures = 4096
numHiddenNodes = 4096
numClasses = 12
featureExtractorName = "alexnet"
trnType = "lstm"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 20 - AlexNet + LSTM + Frame

In [ ]:
numTimesteps = 1
numFeatures = 4096
numHiddenNodes = 4096
numClasses = 12
featureExtractorName = "alexnet"
trnType = "lstm"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 21 - AlexNet + GRU + Chunk

In [ ]:
numTimesteps = 6
numFeatures = 4096
numHiddenNodes = 4096
numClasses = 12
featureExtractorName = "alexnet"
trnType = "gru"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 22 - AlexNet + GRU + Frame

In [ ]:
numTimesteps = 1
numFeatures = 4096
numHiddenNodes = 4096
numClasses = 12
featureExtractorName = "alexnet"
trnType = "gru"
input_dim = (numTimesteps,numFeatures)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 23 - AlexNet + Dense + Chunk

In [ ]:
numTimesteps = 6
numFeatures = 4096
numHiddenNodes = 4096
numClasses = 12
featureExtractorName = "alexnet"
trnType = "dense"
input_dim = (numTimesteps*numFeatures,)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 24 - AlexNet + Dense + Frame

In [ ]:
numTimesteps = 1
numFeatures = 4096
numHiddenNodes = 4096
numClasses = 12
featureExtractorName = "alexnet"
trnType = "dense"
input_dim = (numTimesteps*numFeatures,)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

### Model 25 - End to End AlexNet

In [ ]:
numTimesteps = ''
numFeatures = ''
numHiddenNodes = ''
numClasses = 12
featureExtractorName = ''
trnType = "alexnet"
input_dim = (224,224,3)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

In [ ]:
best_model = load_model("/content/trn__alexnet__0.h5")

partitions = {}
labels = {}
best_fold_index = 0

partitions["validation"], labels["validation"] = DataPartitions.create_partitions_and_labels_cv_val(folds, best_fold_index)
validationGenerator = DataGenerator(partitions["validation"], labels["validation"], dim=input_dim, numClasses=numClasses)

best_model.evaluate(validationGenerator)

### Model 26 - End to End MobileNetV2

In [ ]:
numTimesteps = ''
numFeatures = ''
numHiddenNodes = ''
numClasses = 12
featureExtractorName = ''
trnType = "mobilenet"
input_dim = (224,224,3)

train_model(
    numTimesteps,
    numFeatures,
    numHiddenNodes,
    numClasses,
    featureExtractorName,
    trnType,
    input_dim)

## Confusion Matrix - Model 1

In [ ]:
partitions = {}
labels = {}
best_fold_index = 3

partitions["train"], labels["train"] = DataPartitions.create_partitions_and_labels_cv_train(folds, best_fold_index)
partitions["validation"], labels["validation"] = DataPartitions.create_partitions_and_labels_cv_val(folds, best_fold_index)

trainGenerator = DataGenerator(partitions["train"], labels["train"], dim=input_dim, numClasses=numClasses)
validationGenerator = DataGenerator(partitions["validation"], labels["validation"], dim=input_dim, numClasses=numClasses)

In [ ]:
best_model = load_model("/content/trn_mobilenetv2_lstm_6_3.h5")

In [ ]:
confusion_matrix = {}

for chunk_name, true_label in validationGenerator.labels.items():
    chunk = np.load(chunk_name).reshape((-1,) + input_dim)
    predicted_label = np.argmax(best_model.predict(chunk))
    if true_label not in confusion_matrix:
        confusion_matrix[true_label] = {}
    if predicted_label not in confusion_matrix[true_label]:
        confusion_matrix[true_label][predicted_label] = 0
    confusion_matrix[true_label][predicted_label] += 1

In [ ]:
confusion_matrix

## Confusion Matrix - Model 25

In [ ]:
partitions = {}
labels = {}
best_fold_index = 0

partitions["train"], labels["train"] = DataPartitions.create_partitions_and_labels_cv_train(folds, best_fold_index)
partitions["validation"], labels["validation"] = DataPartitions.create_partitions_and_labels_cv_val(folds, best_fold_index)

trainGenerator = DataGenerator(partitions["train"], labels["train"], dim=input_dim, numClasses=numClasses)
validationGenerator = DataGenerator(partitions["validation"], labels["validation"], dim=input_dim, numClasses=numClasses)

In [ ]:
best_model = load_model("/content/trn__alexnet__0.h5")

In [ ]:
confusion_matrix = {}

for chunk_name, true_label in validationGenerator.labels.items():
    chunk = np.load(chunk_name).reshape((-1,) + input_dim)
    predicted_label = np.argmax(best_model.predict(chunk))
    if true_label not in confusion_matrix:
        confusion_matrix[true_label] = {}
    if predicted_label not in confusion_matrix[true_label]:
        confusion_matrix[true_label][predicted_label] = 0
    confusion_matrix[true_label][predicted_label] += 1

In [ ]:
confusion_matrix